<div style="display: flex; align-items: center; gap: 40px;">

<img src="https://play-lh.googleusercontent.com/_O9p4Z4yucA2NLmZBu9mTJCuBwXeT9NcbtrDN6I8gKlkIPRySV0adOmbyipjSj9Gew" width="130">
<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQtg029pb---rde55bLC81UxjtTfNcVz06_7g&s" width="130">





  <h2>SUTRA by TWO Platforms </h2>
  <p>SUTRA is a family of large multi-lingual language (LMLMs) models pioneered by Two Platforms. SUTRA’s dual-transformer approach extends the power of both MoE and Dense AI language model architectures, delivering cost-efficient multilingual capabilities for over 50+ languages. It powers scalable AI applications for conversation, search, and advanced reasoning, ensuring high-performance across diverse languages, domains and applications.</p>
  </div>



## What is CrewAI?

CrewAI is a framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks through specialized roles and coordinated workflows.


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1eXSKUTYyPJ0QcngMm-aisMikiI2BG58d?usp=sharing)


## Get Your API Keys

Before you begin, make sure you have:

1. A SUTRA API key (Get yours at [TWO AI's SUTRA API page](https://www.two.ai/sutra/api))
2. Basic familiarity with Python and Jupyter notebooks

This notebook is designed to run in Google Colab, so no local Python installation is required.

#SUTRA using CrewAI

### Setup and Installation

First, let's install the required packages:

In [ ]:
!pip install openai crewai==0.120.1  crewai-tools==0.45.0

## Setting up Environment Variables

You'll need to set up your API keys. For security reasons, it's best to use environment variables:

In [2]:
import os
from google.colab import userdata

# Set the API key from Colab secrets
os.environ["SUTRA_API_KEY"] = userdata.get("SUTRA_API_KEY2")

## Basic Usage of SUTRA with OpenAI Client

Let's first see how to use Sutra with the standard OpenAI client:

In [ ]:
from openai import OpenAI

# Initialize the client with Sutra's API endpoint
client = OpenAI(
    base_url='https://api.two.ai/v2',
    api_key=os.environ["SUTRA_API_KEY"]
)


# Simple completion with Sutra
response = client.chat.completions.create(
    model="sutra-v2",  # Use Sutra model
    messages=[
        {"role": "system", "content": "You are a helpful assistant that specializes in Indian languages and culture."},
        {"role": "user", "content": "Tell me about the importance of the Ganga river in Indian culture."}
    ]
)

print(response.choices[0].message.content)

## Integrating SUTRA with CrewAI

Now, let's integrate Sutra with CrewAI to create a team of collaborative agents. First, we need to create a custom LLM class for CrewAI that uses Sutra:

In [4]:
from crewai import LLM

llm = LLM(
    model="openai/sutra-v2",
    api_key=os.getenv("SUTRA_API_KEY"),
    api_base="https://api.two.ai/v2",
)

## Creating a Cultural Research Crew

Let's create a crew of agents that collaborate to research and analyze aspects of Indian culture. We'll define specialized roles for each agent:

In [5]:
from crewai import Agent, Task, Crew, Process , BaseLLM


# Define the agents with specialized roles
cultural_researcher = Agent(
    role="Cultural Research Specialist",
    goal="Research and gather comprehensive information about Indian cultural topics",
    backstory="""You are an expert in Indian history and cultural studies with decades of experience.
    You have deep knowledge of Indian traditions, religions, arts, and social practices across different regions.
    You excel at gathering accurate and nuanced information about cultural topics.""",
    verbose=True,
    llm=llm
)

linguistic_expert = Agent(
    role="Linguistic Expert",
    goal="Analyze and explain language aspects and provide translations when needed",
    backstory="""You are a polyglot specializing in Indian languages including Hindi, Tamil, Bengali, Telugu,
    Marathi, and Punjabi. You can translate between these languages and English, and explain linguistic
    nuances, etymologies, and cultural contexts of words and phrases.""",
    verbose=True,
    llm=llm
)

content_writer = Agent(
    role="Content Writer",
    goal="Create engaging, informative content based on research findings",
    backstory="""You are a talented writer with a gift for making complex cultural topics accessible
    and engaging. You can synthesize research into clear, compelling narratives that respect
    the cultural nuances and authenticity of the subject matter.""",
    verbose=True,
    llm=llm
)

## Defining Tasks for the Crew

Now, let's define specific tasks for each agent in our crew:

In [6]:
# Define the tasks for our cultural research project
research_task = Task(
    description="""Research the significance and history of {topic} in Indian culture.
    Include regional variations, historical evolution, and contemporary relevance.
    Gather facts, stories, and cultural contexts.""",
    expected_output="A comprehensive research report with well-organized facts and cultural insights",
    agent=cultural_researcher
)

linguistic_analysis_task = Task(
    description="""Analyze the linguistic aspects of {topic} in Indian culture.
    Explain key terms in relevant Indian languages, their meanings, and cultural significance.
    Provide translations of important phrases and concepts.""",
    expected_output="A detailed linguistic analysis with translations and cultural context of key terms",
    agent=linguistic_expert
)

content_creation_task = Task(
    description="""Based on the research and linguistic analysis, create an engaging and informative
    article about {topic} in Indian culture. The content should be culturally respectful,
    accurate, and accessible to a global audience while preserving authentic cultural nuances.""",
    expected_output="A well-written, engaging article that effectively communicates the cultural significance of the topic",
    agent=content_writer,
    context=[research_task, linguistic_analysis_task]  # This task depends on the output of the previous tasks
)

## Forming the Crew and Executing Tasks

Now, let's form our crew and execute the tasks in a sequential process:

In [ ]:
# Form the crew with our agents and tasks
cultural_research_crew = Crew(
    agents=[cultural_researcher, linguistic_expert, content_writer],
    tasks=[research_task, linguistic_analysis_task, content_creation_task],
    process=Process.sequential,  # Tasks will be executed in sequence
    verbose=True  # Detailed output of the crew's work
)

# Execute the crew's tasks for a specific cultural topic
result = cultural_research_crew.kickoff(
    inputs={"topic": "Diwali festival"}
)

print("\n\nFinal Result:\n")
print(result)

## Creating a Multilingual Content Creation Crew

Let's create another crew that leverages Sutra's multilingual capabilities to create content in multiple Indian languages:

In [8]:
# Define agents for multilingual content creation
content_strategist = Agent(
    role="Content Strategist",
    goal="Develop effective content strategies for multilingual audiences in India",
    backstory="""You are an experienced content strategist who understands the diverse
    linguistic landscape of India. You know how to tailor content strategies for
    different language communities while maintaining a cohesive brand message.""",
    verbose=True,
    llm=llm
)

hindi_writer = Agent(
    role="Hindi Content Writer",
    goal="Create engaging and culturally relevant content in Hindi",
    backstory="""You are a native Hindi speaker with excellent writing skills. You understand
    the cultural nuances of Hindi-speaking regions and can create content that resonates
    with this audience.""",
    verbose=True,
    llm=llm
)

tamil_writer = Agent(
    role="Tamil Content Writer",
    goal="Create engaging and culturally relevant content in Tamil",
    backstory="""You are a native Tamil speaker with excellent writing skills. You understand
    the cultural nuances of Tamil-speaking regions and can create content that resonates
    with this audience.""",
    verbose=True,
    llm=llm
)

editor = Agent(
    role="Multilingual Editor",
    goal="Ensure consistency and quality across multilingual content",
    backstory="""You are a skilled editor who is fluent in multiple Indian languages.
    You ensure that translations maintain the original message while being culturally
    appropriate for each language audience.""",
    verbose=True,
    llm=llm
)

### Define tasks for multilingual content creation


In [9]:
# Define tasks for multilingual content creation
strategy_task = Task(
    description="""Develop a content strategy for creating multilingual content about {topic}
    for Hindi and Tamil speaking audiences. Consider cultural nuances, regional preferences,
    and effective messaging approaches for each language community.""",
    expected_output="A comprehensive content strategy document with specific guidelines for each language",
    agent=content_strategist
)

hindi_content_task = Task(
    description="""Based on the content strategy, create engaging content in Hindi about {topic}.
    The content should be culturally relevant and resonate with Hindi-speaking audiences.""",
    expected_output="High-quality Hindi content that follows the strategy guidelines",
    agent=hindi_writer,
    context=[strategy_task]
)

tamil_content_task = Task(
    description="""Based on the content strategy, create engaging content in Tamil about {topic}.
    The content should be culturally relevant and resonate with Tamil-speaking audiences.""",
    expected_output="High-quality Tamil content that follows the strategy guidelines",
    agent=tamil_writer,
    context=[strategy_task]
)

editing_task = Task(
    description="""Review and edit the Hindi and Tamil content about {topic}. Ensure consistency
    with the original strategy while preserving the cultural nuances of each language.
    Provide a final assessment of the multilingual content package.""",
    expected_output="Edited versions of both language contents with an assessment report",
    agent=editor,
    context=[strategy_task, hindi_content_task, tamil_content_task]
)

### Form the multilingual content crew


In [ ]:
# Form the multilingual content crew
multilingual_crew = Crew(
    agents=[content_strategist, hindi_writer, tamil_writer, editor],
    tasks=[strategy_task, hindi_content_task, tamil_content_task, editing_task],
    process=Process.sequential,
    verbose=True
)

# Execute the crew's tasks for a specific topic
multilingual_result = multilingual_crew.kickoff(
    inputs={"topic": "sustainable living practices"}
)

print("\n\nFinal Multilingual Content:\n")
print(multilingual_result)

## Conclusion

In this notebook, we've demonstrated how to use the Sutra model with the OpenAI client and integrate it with CrewAI to create collaborative AI agent teams. We've explored:

1. Basic usage of Sutra with the OpenAI client
2. Creating a custom LLM class for CrewAI that uses Sutra
3. Building a cultural research crew with specialized agents
4. Creating a multilingual content creation crew that leverages Sutra's language capabilities

The combination of Sutra's strong multilingual capabilities, especially for Indian languages, with CrewAI's collaborative agent framework provides a powerful platform for building sophisticated AI applications that can understand and respond in multiple languages while leveraging the collective intelligence of specialized agents.